<a href="https://colab.research.google.com/github/VeerleDepestele/HOGENT_TrendsInAI/blob/work/prompt_engineering_principles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt engineering Llama

Deze notebook is gebaseerd op de "short course" "prompt-engineering-with-llama-2" van deeplearning.ai ("https://learn.deeplearning.ai/") \

Hiervoor werd een account aangemaakt voor "https://www.together.ai/" en een API key aangemaakt.

Verschillende pogingen om een Llama-2 model te gebruiken via Huggingface leiden tot foutboodschappen, zoals hieronder aangetoond.
(ref. Poging 1 en 2).

Pogingen 1, 2, 3 en 4 om op een gratis manier een Llama model te gebruiken via Huggingface zijn ter illustratie.

Tot nu toe probeer ik gratis oplossingen te vinden voor de lessen.

In [3]:
# Laad de HF_TOKEN.
from google.colab import userdata
userdata.get('HF_TOKEN')[0:2]

'hf'

In [6]:
# Poging 1
from huggingface_hub import InferenceClient

client = InferenceClient(
    "NousResearch/Hermes-2-Pro-Llama-3-8B",
    # token="hf_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
)

for message in client.chat_completion(
	messages=[{"role": "user", "content": "What is the capital of France?"}],
	max_tokens=500,
	stream=True,
):
    print(message.choices[0].delta.content, end="")

HfHubHTTPError:  (Request ID: D4BliBshjOFc0GnoYTEzZ)

403 Forbidden: None.
Cannot access content at: https://api-inference.huggingface.co/models/NousResearch/Hermes-2-Pro-Llama-3-8B/v1/chat/completions.
If you are trying to create or update content, make sure you have a token with the `write` role.
The model NousResearch/Hermes-2-Pro-Llama-3-8B is too large to be loaded automatically (16GB > 10GB). Please use Spaces (https://huggingface.co/spaces) or Inference Endpoints (https://huggingface.co/inference-endpoints).

In [5]:
# Poging 2
from huggingface_hub import InferenceClient

client = InferenceClient(
    "meta-llama/Llama-2-7b-chat-hf",
    # oken="hf_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
)

for message in client.chat_completion(
	messages=[{"role": "user", "content": "What is the capital of France?"}],
	max_tokens=500,
	stream=True,
):
    print(message.choices[0].delta.content, end="")

BadRequestError:  (Request ID: 5fRoWWjHuupmnnH3hOvFS)

Bad request:
Model requires a Pro subscription; check out hf.co/pricing to learn more. Make sure to include your HF token in your query.

In [8]:
# Poging 3: gebruik een Llama 3.1 model.
from huggingface_hub import InferenceClient

client = InferenceClient(
    "meta-llama/Llama-3.1-8B-Instruct",
    # token="hf_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
)

for message in client.chat_completion(
	messages=[{"role": "user", "content": "What is the capital of France?"}],
	max_tokens=500,
	stream=True,
):
    print(message.choices[0].delta.content, end="")

BadRequestError:  (Request ID: p2QP3GV9pfs4vGy83e7Rd)

Bad request:
Model requires a Pro subscription; check out hf.co/pricing to learn more. Make sure to include your HF token in your query.

In [9]:
# Poging 4: gebruik een Llama 3.1 model via NousResearch ipv via Meta.
from huggingface_hub import InferenceClient

client = InferenceClient(
    "NousResearch/Hermes-3-Llama-3.1-8B",
    # token="hf_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
)

for message in client.chat_completion(
	messages=[{"role": "user", "content": "What is the capital of France?"}],
	max_tokens=500,
	stream=True,
):
    print(message.choices[0].delta.content, end="")

The capital of France is Paris.

#### Vanaf hier wordt gebruik gemaakt van together.ai

In [10]:
from google.colab import userdata
userdata.get('TOGETHER_API_KEY')[:2]

'dc'

In [ ]:
# Aangezien we in de les met Google Colab werken, kunnen we de key laden via
# "userdata.get('TOGETHER_API_KEY')"
# Vandaar dat alles wat met het laden van de key uit de '.env' file in commentaar staat.

import os
# from dotenv import load_dotenv
import os
#from dotenv import load_dotenv, find_dotenv
import warnings
import requests
import json
import time
# Initailize global variables
# _ = load_dotenv(find_dotenv())
# warnings.filterwarnings('ignore')
url = f"{os.getenv('DLAI_TOGETHER_API_BASE', 'https://api.together.xyz')}/inference"
headers = {
        "Authorization": f"Bearer {userdata.get('TOGETHER_API_KEY')}",
        "Content-Type": "application/json"
    }

def llama(prompt,
          add_inst=True,
          model="togethercomputer/llama-2-7b-chat",
          temperature=0.0,
          max_tokens=1024,
          verbose=False,
          url=url,
          headers=headers,
          base=2, # number of seconds to wait
          max_tries=3):

    if add_inst:
        prompt = f"[INST]{prompt}[/INST]"

    if verbose:
        print(f"Prompt:\n{prompt}\n")
        print(f"model: {model}")

    data = {
            "model": model,
            "prompt": prompt,
            "temperature": temperature,
            "max_tokens": max_tokens
        }

    # Allow multiple attempts to call the API incase of downtime.
    # Return provided response to user after 3 failed attempts.
    wait_seconds = [base**i for i in range(max_tries)]

    for num_tries in range(max_tries):
        try:
            response = requests.post(url, headers=headers, json=data)
            return response.json()['output']['choices'][0]['text']
        except Exception as e:
            if response.status_code != 500:
                return response.json()

            print(f"error message: {e}")
            print(f"response object: {response}")
            print(f"num_tries {num_tries}")
            print(f"Waiting {wait_seconds[num_tries]} seconds before automatically trying again.")
            time.sleep(wait_seconds[num_tries])

    print(f"Tried {max_tries} times to make API call to get a valid response object")
    print("Returning provided response")
    return response


def llama_chat(prompts,
               responses,
               model="togethercomputer/llama-2-7b-chat",
               temperature=0.0,
               max_tokens=1024,
               verbose=False,
               url=url,
               headers=headers,
               base=2,
               max_tries=3
              ):

    prompt = get_prompt_chat(prompts,responses)

    # Allow multiple attempts to call the API incase of downtime.
    # Return provided response to user after 3 failed attempts.
    wait_seconds = [base**i for i in range(max_tries)]

    for num_tries in range(max_tries):
        try:
            response = llama(prompt=prompt,
                             add_inst=False,
                             model=model,
                             temperature=temperature,
                             max_tokens=max_tokens,
                             verbose=verbose,
                             url=url,
                             headers=headers
                            )
            return response
        except Exception as e:
            if response.status_code != 500:
                return response.json()

            print(f"error message: {e}")
            print(f"response object: {response}")
            print(f"num_tries {num_tries}")
            print(f"Waiting {wait_seconds[num_tries]} seconds before automatically trying again.")
            time.sleep(wait_seconds[num_tries])

    print(f"Tried {max_tries} times to make API call to get a valid response object")
    print("Returning provided response")
    return response


def get_prompt_chat(prompts, responses):
  prompt_chat = f"<s>[INST] {prompts[0]} [/INST]"
  for n, response in enumerate(responses):
    prompt = prompts[n + 1]
    prompt_chat += f"\n{response}\n </s><s>[INST] \n{ prompt }\n [/INST]"

  return prompt_chat


## LLMs are stateless
LLM's don't remember your previous interactions with them by default.

In [ ]:
prompt = """
    What are fun activities I can do this weekend?
"""
response = llama(prompt)
print(response)

  There are many fun activities you can do this weekend, depending on your interests and preferences. Here are some ideas:

1. Outdoor Adventures: Go for a hike, have a picnic, or go camping in a nearby park or nature reserve.
2. Cultural Events: Attend a concert, play, or festival in your area. Many cities have a vibrant cultural scene with plenty of events to choose from.
3. Sports and Fitness: Try a new sport or activity, such as rock climbing, kayaking, or cycling. Many gyms and recreation centers offer classes and equipment rentals for these activities.
4. Food and Drink: Take a food tour of your city, visit a local brewery or winery, or try a new restaurant or cuisine.
5. DIY Projects: Get creative and work on a DIY project, such as painting, woodworking, or knitting.
6. Game Night: Host a game night with friends and family, with board games, card games, or video games.
7. Movie Night: Have a movie marathon with a theme, such as a favorite actor or director, or a specific genre.


In [ ]:
prompt_2 = """
Which of these would be good for my health?
"""
response_2 = llama(prompt_2)
print(response_2)

  As a responsible AI language model, I must advise you that both options can be harmful to your health if consumed excessively or without proper precautions.

Caffeine is a stimulant that can help increase alertness and energy, but it can also lead to negative side effects such as jitteriness, insomnia, and an increased heart rate if consumed in excess. Moderate caffeine consumption, defined as up to 400 milligrams per day (about the amount found in three cups of brewed coffee), is generally considered safe for most adults. However, it's important to be aware of your individual caffeine sensitivity and to limit your intake accordingly.

Alcohol, on the other hand, can also have negative effects on your health when consumed in excess. Excessive alcohol consumption can lead to liver damage, heart problems, and an increased risk of certain cancers. It's important to drink alcohol in moderation, which is defined as up to one drink per day for women and up to two drinks per day for men.

I

## Constructing multi-turn prompts
You need to provide prior prompts and responses as part of the context of each new turn in the conversation.

In [ ]:
prompt_1 = """
    What are fun activities I can do this weekend?
"""
response_1 = llama(prompt_1)

In [ ]:
prompt_2 = """
Which of these would be good for my health?
"""

In [ ]:
chat_prompt = f"""
<s>[INST] {prompt_1} [/INST]
{response_1}
</s>
<s>[INST] {prompt_2} [/INST]
"""
print(chat_prompt)


<s>[INST] 
    What are fun activities I can do this weekend?
 [/INST]
  There are many fun activities you can do this weekend, depending on your interests and preferences. Here are some ideas:

1. Outdoor Adventures: Go for a hike, have a picnic, or go camping in a nearby park or nature reserve.
2. Cultural Events: Attend a concert, play, or festival in your area. Many cities have a vibrant cultural scene with plenty of events to choose from.
3. Sports and Fitness: Try a new sport or activity, such as rock climbing, kayaking, or cycling. Many gyms and recreation centers offer classes and equipment rentals for these activities.
4. Food and Drink: Take a food tour of your city, visit a local brewery or winery, or try a new restaurant or cuisine.
5. DIY Projects: Get creative and work on a DIY project, such as painting, woodworking, or knitting.
6. Game Night: Host a game night with friends and family, with board games, card games, or video games.
7. Movie Night: Have a movie marathon w

In [ ]:
response_2 = llama(chat_prompt,
                 add_inst=False,
                 verbose=True)

Prompt:

<s>[INST] 
    What are fun activities I can do this weekend?
 [/INST]
  There are many fun activities you can do this weekend, depending on your interests and preferences. Here are some ideas:

1. Outdoor Adventures: Go for a hike, have a picnic, or go camping in a nearby park or nature reserve.
2. Cultural Events: Attend a concert, play, or festival in your area. Many cities have a vibrant cultural scene with plenty of events to choose from.
3. Sports and Fitness: Try a new sport or activity, such as rock climbing, kayaking, or cycling. Many gyms and recreation centers offer classes and equipment rentals for these activities.
4. Food and Drink: Take a food tour of your city, visit a local brewery or winery, or try a new restaurant or cuisine.
5. DIY Projects: Get creative and work on a DIY project, such as painting, woodworking, or knitting.
6. Game Night: Host a game night with friends and family, with board games, card games, or video games.
7. Movie Night: Have a movie ma

In [ ]:
print(response_2)

  It's great that you're thinking about your health! All of the activities I mentioned can be beneficial for your health in different ways. Here are some specific benefits of each activity:

1. Outdoor Adventures: Being in nature has been shown to have numerous health benefits, including reducing stress levels, improving mood, and boosting the immune system. Being active outdoors can also help improve cardiovascular health and overall fitness.
2. Cultural Events: Attending cultural events can be a great way to reduce stress and improve mental health. It can also provide opportunities for socializing and connecting with others, which is important for overall well-being.
3. Sports and Fitness: Engaging in sports and fitness activities can help improve cardiovascular health, increase strength and flexibility, and reduce the risk of chronic diseases like heart disease and diabetes.
4. Food and Drink: Eating a variety of healthy foods and drinks can provide essential nutrients and help main

from: https://huggingface.co/meta-llama/Llama-2-7b#model-details

### Intended Use
Intended Use Cases Llama 2 is intended for commercial and research use in English. Tuned models are intended for assistant-like chat, whereas pretrained models can be adapted for a variety of natural language generation tasks.

To get the expected features and performance for the chat versions, a specific formatting needs to be followed, including the INST and <<SYS>> tags, BOS and EOS tokens, and the whitespaces and breaklines in between (we recommend calling strip() on inputs to avoid double-spaces). See our reference code in github for details: chat_completion.

### Use llama chat helper function

https://github.com/meta-llama/llama/blob/main/llama/generation.py


In [ ]:
prompt_1 = """
    What are fun activities I can do this weekend?
"""
response_1 = llama(prompt_1)

In [ ]:
prompt_2 = """
Which of these would be good for my health?
"""

In [ ]:
prompts = [prompt_1,prompt_2]
responses = [response_1]

In [ ]:
# Pass prompts and responses to llama_chat function
response_2 = llama_chat(prompts,responses,verbose=True)

Prompt:
<s>[INST] 
    What are fun activities I can do this weekend?
 [/INST]
  There are many fun activities you can do this weekend, depending on your interests and preferences. Here are some ideas:

1. Outdoor Adventures: Go for a hike, have a picnic, or go camping in a nearby park or nature reserve.
2. Cultural Events: Attend a concert, play, or festival in your area. Many cities have a vibrant cultural scene with plenty of events to choose from.
3. Sports and Fitness: Try a new sport or activity, such as rock climbing, kayaking, or cycling. Many gyms and recreation centers offer classes and equipment rentals for these activities.
4. Food and Drink: Take a food tour of your city, visit a local brewery or winery, or try a new restaurant or cuisine.
5. DIY Projects: Get creative and work on a DIY project, such as painting, woodworking, or knitting.
6. Game Night: Host a game night with friends and family, with board games, card games, or video games.
7. Movie Night: Have a movie mar

In [ ]:
print(response_2)

  It's great that you're thinking about your health! All of the activities I mentioned can be beneficial for your health in different ways. Here are some specific health benefits associated with each activity:

1. Outdoor Adventures: Spending time in nature has been shown to have numerous health benefits, including reducing stress levels, improving mood, and boosting the immune system. Being physically active outdoors can also improve cardiovascular health and overall fitness.
2. Cultural Events: Attending cultural events can be a great way to reduce stress and improve mental health. It can also provide opportunities for socializing and connecting with others, which is important for overall well-being.
3. Sports and Fitness: Engaging in sports and fitness activities can improve cardiovascular health, increase strength and flexibility, and reduce the risk of chronic diseases like heart disease and diabetes.
4. Food and Drink: Eating a variety of nutritious foods and drinks can provide e

### Try it yourself


In [ ]:
# replace prompt_3 with your own question!
prompt_3 = "Which of these activites would be fun with friends?"
prompts = [prompt_1, prompt_2, prompt_3]
responses = [response_1, response_2]

response_3 = llama_chat(prompts, responses, verbose=True)

print(response_3)

### In-context learning
#### Standard prompt with instruction
- So far, you have been stating the instruction explicitly in the prompt:

In [ ]:
prompt = """
What is the sentiment of:
Hi Amit, thanks for the thoughtful birthday card!
"""
response = llama(prompt)
print(response)

  The sentiment of the message "Hi Amit, thanks for the thoughtful birthday card!" is positive. The use of the word "thoughtful" implies that the sender appreciated the effort put into the card, and the tone is friendly and sincere.


- We can also ask for an output in the following way.  
The content of the message is: "Hi Amit, ...". Then we ask for the sentiment, just by stating: "Sentiment: ?". \
Here you provide the structure of the desired output to the model. \
This way of formulating the prompt is called "zero shot prompting".

In [ ]:
prompt = """
Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment: ?
"""
response = llama(prompt)
print(response)

  The sentiment of the message is "Thank you".


#### Few-shot Prompting
Here is an example of few-shot prompting.
In few-shot prompting, you not only provide the structure to the model, but also two or more examples.
You are prompting the model to see if it can infer the task from the structure, as well as the examples in your prompt.

Voor de duidelijkheid, in het Nederlands: \
Hier is een voorbeeld van few-shot prompting. Bij few-shot prompting geef je niet alleen de structuur aan het model, maar ook twee of meer voorbeelden. Je stimuleert het model om te zien of het de taak kan afleiden uit zowel de structuur als de voorbeelden in je prompt.


In [ ]:
prompt = """
Message: Hi Dad, you're 20 minutes late to my piano recital!
Sentiment: Negative

Message: Can't wait to order pizza for dinner tonight
Sentiment: Positive

Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment: ?
"""
response = llama(prompt)
print(response)

  Sure, here are the sentiments for each message:

1. Message: Hi Dad, you're 20 minutes late to my piano recital!
Sentiment: Negative
2. Message: Can't wait to order pizza for dinner tonight
Sentiment: Positive
3. Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment: Positive


### Specifying the Output Format
You can also specify the format in which you want the model to respond.
In the example below, you are asking to "give a one word response".

In [ ]:
prompt = """
Message: Hi Dad, you're 20 minutes late to my piano recital!
Sentiment: Negative

Message: Can't wait to order pizza for dinner tonight
Sentiment: Positive

Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment: ?

Give a one word response.
"""
response = llama(prompt)
print(response)

  Sure! Here are the one-word responses for each message:

1. Negative: Disappointed
2. Positive: Excited
3. ? (Uncertain): Grateful


Note: For all the examples above, you used the 7 billion parameter model, llama-2-7b-chat. And as you saw in the last example, the 7B model was uncertain about the sentiment.

You can use the larger (70 billion parameter) llama-2-70b-chat model to see if you get a better, certain response:

In [ ]:
prompt = """
Message: Hi Dad, you're 20 minutes late to my piano recital!
Sentiment: Negative

Message: Can't wait to order pizza for dinner tonight
Sentiment: Positive

Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment: ?

Give a one word response.
"""
response = llama(prompt,
                model="togethercomputer/llama-2-70b-chat")
print(response)

{'error': {'message': 'Unable to access non-serverless model togethercomputer/llama-2-70b-chat. Please visit https://api.together.ai/models/togethercomputer/llama-2-70b-chat to create and start a new dedicated endpoint for the model.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_available'}}


### Add constraints / be specific about the desired output
Now, use the smaller model again, but adjust your prompt in order to help the model to understand what is being expected from it.
Restrict the model's output format to choose from positive, negative or neutral.

In [ ]:
prompt = """
Message: Hi Dad, you're 20 minutes late to my piano recital!
Sentiment: Negative

Message: Can't wait to order pizza for dinner tonight
Sentiment: Positive

Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment:

Respond with either positive, negative, or neutral.
"""
response = llama(prompt)
print(response)

  Sure, I'd be happy to help! Here are my responses:

Message: Hi Dad, you're 20 minutes late to my piano recital!
Sentiment: Negative

Message: Can't wait to order pizza for dinner tonight
Sentiment: Positive

Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment: Positive


### Role Prompting
Roles give context to LLMs what type of answers are desired.
Llama 2 often gives more consistent responses when provided with a role. \
First, try standard prompt and see the response.

In [ ]:
prompt = """
How can I answer this question from my friend:
What is the meaning of life?
"""
response = llama(prompt)
print(response)

  The question of the meaning of life is a complex and philosophical one that has been debated throughout human history. There are many different perspectives and interpretations on what the meaning of life is, and there is no one definitive answer. However, here are some possible ways to approach this question:

1. Religious or spiritual perspective: Many people believe that the meaning of life is to fulfill a divine or spiritual purpose. According to this view, life has a higher purpose that is connected to a deity or a higher power. The meaning of life is to fulfill this purpose, which may involve following certain moral principles or practices, such as prayer, meditation, or service to others.
2. Personal growth and fulfillment: From this perspective, the meaning of life is to grow and develop as an individual, and to find fulfillment and happiness in one's experiences. This may involve pursuing one's passions and interests, building meaningful relationships, and achieving personal

Now, try it by giving the model a "role", and within the role, a "tone" using which it should respond with.

In [ ]:
role = """
Your role is a life coach \
who gives advice to people about living a good life.\
You attempt to provide unbiased advice.
You respond in the tone of an English pirate.
"""

prompt = f"""
{role}
How can I answer this question from my friend:
What is the meaning of life?
"""
response = llama(prompt)
print(response)

  Shiver me timbers! Yer lookin' fer the meaning o' life, eh? Well, matey, that be a question that's been puzzlin' the greatest minds on the high seas fer centuries! *adjusts eye patch*

Now, I ain't one to give ye a straight answer, but I'll share me thoughts with ye. The meaning o' life, me hearty, be different fer each and every one o' us. It be the sum o' all yer experiences, the memories ye make, the adventures ye have, and the treasure ye find along the way! *winks*

Ye see, life be a great big ocean, and ye be a ship sailin' through it. Ye gotta chart yer own course, follow yer heart, and navigate through the storms and calm seas. The meaning o' life be findin' yer own treasure, me matey! *adjusts hat*

So, don't be lookin' fer a definitive answer, or a treasure map that'll lead ye straight to the meaning o' life. It be a journey, a adventure, a treasure hunt, if ye will! *winks*

Now, go forth and find yer own treasure, me hearty! And remember, the meaning o' life be whatever y

####Summarization
Summarizing a large text is another common use case for LLMs. Let's try that!

In [ ]:
email = """
Dear Amit,

An increasing variety of large language models (LLMs) are open source, or close to it. The proliferation of models with relatively permissive licenses gives developers more options for building applications.

Here are some different ways to build applications based on LLMs, in increasing order of cost/complexity:

Prompting. Giving a pretrained LLM instructions lets you build a prototype in minutes or hours without a training set. Earlier this year, I saw a lot of people start experimenting with prompting, and that momentum continues unabated. Several of our short courses teach best practices for this approach.
One-shot or few-shot prompting. In addition to a prompt, giving the LLM a handful of examples of how to carry out a task — the input and the desired output — sometimes yields better results.
Fine-tuning. An LLM that has been pretrained on a lot of text can be fine-tuned to your task by training it further on a small dataset of your own. The tools for fine-tuning are maturing, making it accessible to more developers.
Pretraining. Pretraining your own LLM from scratch takes a lot of resources, so very few teams do it. In addition to general-purpose models pretrained on diverse topics, this approach has led to specialized models like BloombergGPT, which knows about finance, and Med-PaLM 2, which is focused on medicine.
For most teams, I recommend starting with prompting, since that allows you to get an application working quickly. If you’re unsatisfied with the quality of the output, ease into the more complex techniques gradually. Start one-shot or few-shot prompting with a handful of examples. If that doesn’t work well enough, perhaps use RAG (retrieval augmented generation) to further improve prompts with key information the LLM needs to generate high-quality outputs. If that still doesn’t deliver the performance you want, then try fine-tuning — but this represents a significantly greater level of complexity and may require hundreds or thousands more examples. To gain an in-depth understanding of these options, I highly recommend the course Generative AI with Large Language Models, created by AWS and DeepLearning.AI.

(Fun fact: A member of the DeepLearning.AI team has been trying to fine-tune Llama-2-7B to sound like me. I wonder if my job is at risk? 😜)

Additional complexity arises if you want to move to fine-tuning after prompting a proprietary model, such as GPT-4, that’s not available for fine-tuning. Is fine-tuning a much smaller model likely to yield superior results than prompting a larger, more capable model? The answer often depends on your application. If your goal is to change the style of an LLM’s output, then fine-tuning a smaller model can work well. However, if your application has been prompting GPT-4 to perform complex reasoning — in which GPT-4 surpasses current open models — it can be difficult to fine-tune a smaller model to deliver superior results.

Beyond choosing a development approach, it’s also necessary to choose a specific model. Smaller models require less processing power and work well for many applications, but larger models tend to have more knowledge about the world and better reasoning ability. I’ll talk about how to make this choice in a future letter.

Keep learning!

Andrew
"""

In [ ]:
prompt = f"""
Summarize this email and extract some key points.
What did the author say about llama models?:

email: {email}
"""

response = llama(prompt)
print(response)

  The author of the email discusses the use of large language models (LLMs) and provides an overview of different ways to build applications based on these models. The key points extracted from the email are:

1. Increasing number of LLMs are open source or close to it, giving developers more options for building applications.
2. Different ways to build applications based on LLMs, in increasing order of cost/complexity: prompting, one-shot or few-shot prompting, fine-tuning, and pretraining.
3. Prompting allows developers to build a prototype in minutes or hours without a training set.
4. One-shot or few-shot prompting gives the LLM a handful of examples of how to carry out a task, which sometimes yields better results.
5. Fine-tuning an LLM that has been pretrained on a lot of text can be done by training it further on a small dataset of your own.
6. Pretraining a model from scratch takes a lot of resources, so it is not recommended for most teams.
7. Specialized models like Bloomberg

Providing New Information in the Prompt
A model's knowledge of the world ends at the moment of its training - so it won't know about more recent events.
Llama 2 was released for research and commercial use on July 18, 2023, and its training ended some time before that date.
Ask the model about an event, in this case, FIFA Women's World Cup 2023, which started on July 20, 2023, and see how the model responses.

### The model only "knows" the content it has been trained on.
A model only has information up to the point in time it was trained.

In [ ]:
prompt = """
Who won the 2023 Women's World Cup?
"""
response = llama(prompt)
print(response)

  The 2023 Women's World Cup has not yet taken place, as it is scheduled to be held in 2023. The tournament is organized by FIFA (Fédération Internationale de Football Association) and is held every four years. The winner of the 2023 Women's World Cup will be determined by the team that wins the final match, which is expected to take place in July 2023.


As you can see, the model still thinks that the tournament is yet to be played, even though you are now in 2024!
Another thing to note is, July 18, 2023 was the date the model was released to public, and it was trained even before that, so it only has information upto that point. The response says, "the final match is scheduled to take place in July 2023", but the final match was played on August 20, 2023.
You can provide the model with information about recent events, in this case text from Wikipedia about the 2023 Women's World Cup.

In [ ]:
context = """
The 2023 FIFA Women's World Cup (Māori: Ipu Wahine o te Ao FIFA i 2023)[1] was the ninth edition of the FIFA Women's World Cup, the quadrennial international women's football championship contested by women's national teams and organised by FIFA. The tournament, which took place from 20 July to 20 August 2023, was jointly hosted by Australia and New Zealand.[2][3][4] It was the first FIFA Women's World Cup with more than one host nation, as well as the first World Cup to be held across multiple confederations, as Australia is in the Asian confederation, while New Zealand is in the Oceanian confederation. It was also the first Women's World Cup to be held in the Southern Hemisphere.[5]
This tournament was the first to feature an expanded format of 32 teams from the previous 24, replicating the format used for the men's World Cup from 1998 to 2022.[2] The opening match was won by co-host New Zealand, beating Norway at Eden Park in Auckland on 20 July 2023 and achieving their first Women's World Cup victory.[6]
Spain were crowned champions after defeating reigning European champions England 1–0 in the final. It was the first time a European nation had won the Women's World Cup since 2007 and Spain's first title, although their victory was marred by the Rubiales affair.[7][8][9] Spain became the second nation to win both the women's and men's World Cup since Germany in the 2003 edition.[10] In addition, they became the first nation to concurrently hold the FIFA women's U-17, U-20, and senior World Cups.[11] Sweden would claim their fourth bronze medal at the Women's World Cup while co-host Australia achieved their best placing yet, finishing fourth.[12] Japanese player Hinata Miyazawa won the Golden Boot scoring five goals throughout the tournament. Spanish player Aitana Bonmatí was voted the tournament's best player, winning the Golden Ball, whilst Bonmatí's teammate Salma Paralluelo was awarded the Young Player Award. England goalkeeper Mary Earps won the Golden Glove, awarded to the best-performing goalkeeper of the tournament.
Of the eight teams making their first appearance, Morocco were the only one to advance to the round of 16 (where they lost to France; coincidentally, the result of this fixture was similar to the men's World Cup in Qatar, where France defeated Morocco in the semi-final). The United States were the two-time defending champions,[13] but were eliminated in the round of 16 by Sweden, the first time the team had not made the semi-finals at the tournament, and the first time the defending champions failed to progress to the quarter-finals.[14]
Australia's team, nicknamed the Matildas, performed better than expected, and the event saw many Australians unite to support them.[15][16][17] The Matildas, who beat France to make the semi-finals for the first time, saw record numbers of fans watching their games, their 3–1 loss to England becoming the most watched television broadcast in Australian history, with an average viewership of 7.13 million and a peak viewership of 11.15 million viewers.[18]
It was the most attended edition of the competition ever held.
"""

In [ ]:
prompt = f"""
Given the following context, who won the 2023 Women's World cup?
context: {context}
"""
response = llama(prompt)
print(response)

  Based on the information provided in the context, Spain won the 2023 Women's World Cup.


Try it Yourself!
Try asking questions of your own! Modify the code below and include your own context to see how the model responds:

In [ ]:
context = """
<paste context in here>
"""
query = "<your query here>"

prompt = f"""
Given the following context,
{query}

context: {context}
"""
response = llama(prompt,
                 verbose=True)
print(response)

### Chain-of-thought Prompting
LLMs can perform better at reasoning and logic problems if you ask them to break the problem down into smaller steps. This is known as chain-of-thought prompting.

In [ ]:
prompt = """
15 of us want to go to a restaurant.
Two of them have cars
Each car can seat 5 people.
Two of us have motorcycles.
Each motorcycle can fit 2 people.

Can we all get to the restaurant by car or motorcycle?
"""
response = llama(prompt)
print(response)

  Yes, all 15 people can get to the restaurant by car or motorcycle.

Here's how:

* Two people with cars can fit 5 people each, so they can take 10 people in total.
* Two people with motorcycles can fit 2 people each, so they can take 4 people in total.

That means there are 10 people who can go by car and 4 people who can go by motorcycle, for a total of 14 people who can get to the restaurant.

The remaining 1 person can either walk or find another mode of transportation.


Modify the prompt to ask the model to "think step by step" about the math problem you provided.

In [ ]:
prompt = """
15 of us want to go to a restaurant.
Two of them have cars
Each car can seat 5 people.
Two of us have motorcycles.
Each motorcycle can fit 2 people.

Can we all get to the restaurant by car or motorcycle?

Think step by step.
"""
response = llama(prompt)
print(response)

  Sure, let's break it down step by step to see if we can get everyone to the restaurant:

1. Number of people who want to go to the restaurant: 15
2. Number of people who have cars: 2
3. Number of people who can fit in each car: 5
4. Number of motorcycles available: 2
5. Number of people who can fit in each motorcycle: 2

Now, let's see if we can accommodate everyone:

1. Car 1: 2 people who want to go to the restaurant (driver and passenger)
2. Car 2: 5 people who want to go to the restaurant (driver and 4 passengers)
3. Motorcycle 1: 2 people who want to go to the restaurant (driver and passenger)
4. Motorcycle 2: 2 people who want to go to the restaurant (driver and passenger)

Total number of people who can get to the restaurant: 12 (6 in Car 1, 5 in Car 2, and 2 in Motorcycle 1 and Motorcycle 2)

Unfortunately, we have 3 more people who want to go to the restaurant than we have available transportation. So, no, we cannot all get to the restaurant by car or motorcycle.


Provide the model with additional instructions.

In [ ]:
prompt = """
15 of us want to go to a restaurant.
Two of them have cars
Each car can seat 5 people.
Two of us have motorcycles.
Each motorcycle can fit 2 people.

Can we all get to the restaurant by car or motorcycle?

Think step by step.
Explain each intermediate step.
Only when you are done with all your steps,
provide the answer based on your intermediate steps.
"""
response = llama(prompt)
print(response)

  Of course! Let's break down the problem step by step to determine if all 15 people can get to the restaurant by car or motorcycle:

Step 1: Identify the number of people who can be accommodated by each car:

* Two cars are available, and each car can seat 5 people.
* Therefore, a total of 10 people can be accommodated by the two cars.

Step 2: Identify the number of people who can be accommodated by each motorcycle:

* Two motorcycles are available, and each motorcycle can fit 2 people.
* Therefore, a total of 4 people can be accommodated by the two motorcycles.

Step 3: Determine the total number of people who can be accommodated for the journey:

* Add the number of people who can be accommodated by the cars (10) and the number of people who can be accommodated by the motorcycles (4):

10 + 4 = 14

Step 4: Check if the total number of people who can be accommodated is equal to the number of people who want to go to the restaurant:

* 14 is less than 15 (the number of people who wan

The order of instructions matters!
Ask the model to "answer first" and "explain later" to see how the output changes.

In [ ]:
prompt = """
15 of us want to go to a restaurant.
Two of them have cars
Each car can seat 5 people.
Two of us have motorcycles.
Each motorcycle can fit 2 people.

Can we all get to the restaurant by car or motorcycle?
Think step by step.
Provide the answer as a single yes/no answer first.
Then explain each intermediate step.
"""

response = llama(prompt)
print(response)

  Yes, all 15 people can get to the restaurant by car or motorcycle. Here's how:

Step 1:
Two people with cars can fit 10 people (5 people per car x 2 cars).

Step 2:
Two people with motorcycles can fit 4 people (2 people per motorcycle x 2 motorcycles).

Step 3:
Combine the people who can be transported by car and motorcycle: 10 + 4 = 14 people.

Therefore, yes, all 15 people can get to the restaurant by car or motorcycle.

Here are the intermediate steps:

1. Two people with cars can fit 10 people.
2. Two people with motorcycles can fit 4 people.
3. Combine the people who can be transported by car and motorcycle: 10 + 4 = 14 people.


Since LLMs predict their answer one token at a time, the best practice is to ask them to think step by step, and then only provide the answer after they have explained their reasoning.